In [1]:
import os,re
import pandas as pd

PROJECT="TCGA_methylation"
GENERAL="/users/genomics/marta"
DIR=os.path.join(GENERAL,PROJECT)

try:
    os.makedirs(DIR) #path where to store all the itermediate steps and outputs of the pipeline
except:
    print("Directory for %s already exists" %PROJECT)

DATASETDIR="/datasets/marta/TCGA/MET_matched_cancers_biospecimen" # path where samples are saved

GENOMEDIR="/genomics/users/marta/genomes"

try:
    os.makedirs(os.path.join(DIR,"analysis"))
    os.makedirs(os.path.join(DIR,"results"))
    #os.makedirs(os.path.join(DIR,"scripts"))
except:
    print("Directory exists")



Directory for TCGA_methylation already exists
Directory exists


Generates a file looking like normal - tumor - patient id

In [2]:
input = pd.read_csv(os.path.join(DATASETDIR,"clinical_merged_modified_bothRNAMET.tsv"))

df = input[['Project ID','File ID', 'caseID', 'Sample Type', 'gender', 'age_at_index']]
df.drop_duplicates(inplace=True)

normal = df[df['Sample Type'] == "Solid Tissue Normal"]
normal.drop(['Sample Type'], axis=1, inplace=True)
normal.rename(columns={'File ID':'normal_ID', 'caseID':'case_ID'}, inplace=True)

tumor = df[df['Sample Type'] == "Primary Tumor"]
tumor.drop(['Sample Type'], axis=1, inplace=True)
tumor.rename(columns={'File ID':'tumor_ID', 'caseID':'case_ID'}, inplace=True)

# merge
patients = pd.merge(normal, tumor, on=['Project ID','case_ID','gender','age_at_index'])
patients = patients[['Project ID','case_ID', 'normal_ID','tumor_ID', 'gender', 'age_at_index']]
patients.drop_duplicates(inplace=True)
patients.to_csv(os.path.join(DATASETDIR,"samples/patients.csv"), index=False)
patients.to_csv(os.path.join(GENERAL,PROJECT,"results/patients.csv"), index=False)

/tmp/ipykernel_4173465/3330414236.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
/tmp/ipykernel_4173465/3330414236.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal.drop(['Sample Type'], axis=1, inplace=True)
/tmp/ipykernel_4173465/3330414236.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal.rename(columns={'File ID':'normal_ID', 'caseID':'case_ID'}, inplace=True)
/tmp/ipykernel_4

Save beta values ans structure the folder knowing everytime the cancer type, the patient ID and the sample type

In [3]:
cancertypes = patients[['Project ID','case_ID']].drop_duplicates()
cancertypes_dict = cancertypes.set_index('case_ID')['Project ID'].to_dict()

normals = patients[['case_ID','normal_ID']].drop_duplicates()
normals_dict = normals.set_index('normal_ID')['case_ID'].to_dict()

tumors = patients[['case_ID','tumor_ID']].drop_duplicates()
tumors_dict = tumors.set_index('tumor_ID')['case_ID'].to_dict()

In [4]:
patients_folders = [x for x in os.listdir(os.path.join(DATASETDIR, "samples")) if os.path.isdir(os.path.join(DATASETDIR, "samples", x))]
print(len(patients_folders))

869


In [55]:
for folder in patients_folders:
    # know if it is a tumor or a normal sample
    if folder in normals_dict.keys():
        patientID = normals_dict[folder]
        patientID_sample = patientID + "_normal"
    elif folder in tumors_dict.keys():
        patientID = tumors_dict[folder]
        patientID_sample = patientID + "_tumor"
    else:
        next
    # get the cancer type and create a directory
    cancer = cancertypes_dict[patientID]
    try:
        os.makedirs(os.path.join(GENERAL,PROJECT,"analysis/0_bvalues",cancer))
    except:
        next


    print(cancer)
    print(patientID_sample)

    for file in os.listdir(os.path.join(DATASETDIR, "samples",folder)):
        if file.endswith("level3betas.txt"):
            betavalues_downloaded = pd.read_csv(os.path.join(DATASETDIR, "samples",folder,file), header=None, names=['CpG','Bvalue'], sep="\t")
    # there are some patients with more than 1 tumor sample, if that is the case, add a _2 after the patientID
    outfile = os.path.join(GENERAL,PROJECT,"analysis/0_bvalues",cancer)+"/"+patientID_sample+"_betavalues.txt"
    if os.path.isfile(outfile):
        outfile = os.path.join(GENERAL,PROJECT,"analysis/0_bvalues",cancer)+"/"+patientID_sample+"_2_betavalues.txt"
    betavalues_downloaded.to_csv(outfile, sep="\t", index=None)

TCGA-LUSC
TCGA-22-5478_tumor
TCGA-UCEC
TCGA-DI-A2QY_tumor
TCGA-KIRC
TCGA-CZ-5461_normal
TCGA-UCEC
TCGA-BK-A13C_normal
TCGA-HNSC
TCGA-CV-7103_normal
TCGA-UCEC
TCGA-E6-A1M0_normal
TCGA-KIRC
TCGA-CZ-5468_tumor
TCGA-BRCA
TCGA-BH-A1FM_tumor
TCGA-HNSC
TCGA-CV-6959_tumor
TCGA-BRCA
TCGA-BH-A18Q_tumor
TCGA-ESCA
TCGA-L5-A4OQ_tumor
TCGA-THCA
TCGA-BJ-A28R_tumor
TCGA-LIHC
TCGA-DD-A39Z_normal
TCGA-LUAD
TCGA-44-6148_normal
TCGA-KIRC
TCGA-CZ-5468_normal
TCGA-LIHC
TCGA-FV-A2QR_tumor
TCGA-BRCA
TCGA-BH-A1FM_normal
TCGA-KIRP
TCGA-BQ-7046_tumor
TCGA-BRCA
TCGA-BH-A0BW_tumor
TCGA-BRCA
TCGA-E9-A1RI_normal
TCGA-THCA
TCGA-EM-A1YC_tumor
TCGA-BRCA
TCGA-BH-A18L_tumor
TCGA-COAD
TCGA-AA-3713_normal
TCGA-THCA
TCGA-EL-A3T0_tumor
TCGA-BRCA
TCGA-BH-A18R_normal
TCGA-UCEC
TCGA-BG-A2AD_normal
TCGA-BRCA
TCGA-BH-A0DP_tumor
TCGA-ESCA
TCGA-L5-A4OQ_normal
TCGA-KIRP
TCGA-BQ-5884_normal
TCGA-COAD
TCGA-A6-2686_normal
TCGA-PRAD
TCGA-EJ-7125_tumor
TCGA-PAAD
TCGA-H6-8124_normal
TCGA-PRAD
TCGA-HC-7752_normal
TCGA-ESCA
TCGA-L5-A43C_tum

Convert Bvalues to Mvalues per cancer type

In [22]:
%%bash -s "$GENERAL" "$PROJECT"

mkdir $1/$2/analysis/1_mvalues
module load R/4.1.2-foss-2020b

for ctype_path in $1/$2/analysis/0_bvalues/*; do
    # get cancer type
    ctype=$(basename "$ctype_path" | cut -c -9)
    echo $ctype

    # Rscript ~/PycharmProjects/neoantigens/methylation_TCGA/scripts/B2Mvalues.R $ctype_path $1/$2/analysis/1_mvalues/$ctype
    Rscript scripts/B2Mvalues.R $ctype_path $1/$2/analysis/1_mvalues/$ctype
done

mkdir: cannot create directory ‘/users/genomics/marta/TCGA_methylation/analysis/1_mvalues’: File exists


TCGA-BLCA



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

Loading required package: Biobase
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tappl

TCGA-UCEC



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

Loading required package: Biobase
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tappl

...


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

Loading required package: Biobase
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tappl

TCGA-THYM



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

Loading required package: Biobase
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tappl

TCGA-UCEC



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

Loading required package: Biobase
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tappl

Obtain only 1 file per cancer type both per Mvalues and Bvalues

In [ ]:
%%bash -s "$GENERAL" "$PROJECT"

module load R/4.1.2-foss-2020b

for ctype_path in $1/$2/analysis/*values/*; do
    # get cancer type
    ctype=$(basename "$ctype_path" | cut -c -9)
    echo $ctype

    # Rscript ~/PycharmProjects/neoantigens/methylation_TCGA/scripts/B2Mvalues.R $ctype_path $1/$2/analysis/1_mvalues/$ctype
    Rscript scripts/oneFile_CancerType.R $ctype_path
done

TCGA-BLCA



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

